In [1]:
import os
import time
os.chdir("../")
from dataclasses import dataclass
from pathlib import Path
from zipfile import ZipFile
from DeepClassifier.constants import *
from DeepClassifier.utils import read_yaml, create_directories
from DeepClassifier.entities import DataIngestionConfig
import tensorflow as tf

In [2]:
@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

## Configuration Manager

In [3]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path: Path = CONFIG_FILE_PATH,
        params_file_path: Path = PARAMS_FILE_PATH
    ) -> None:
        """Inits ConfigurationManager.

        Args:
            config_file_path (Path, optional): Path of the config.yaml file.
                Defaults to the constant CONFIG_FILE_PATH.
            params_file_path (Path, optional): Path of the params.yaml file.
                Defaults to the constant PARAMS_FILE_PATH.
        """
        # Getting information in the config.yaml and params.yaml file
        self.config = read_yaml(yaml_file_path=config_file_path)
        self.params = read_yaml(yaml_file_path=params_file_path)
        
        # Creating the 'artifacts' directory
        create_directories(paths_of_directories=[self.config.artifacts_root])
    
    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        """Creates and returns PrepareCallbacksConfig.

        Returns:
            PrepareCallbacksConfig: The PrepareCallbacksConfig.
        """
        # Getting the values in the `prepare_callbacks` key of the config.yaml
        # file
        config = self.config.prepare_callbacks
        
        # Creating the directories 'artifacts/prepare_callbacks',
        # 'artifacts/prepare_callbacks/tensorboard_logs', and
        # 'artifacts/prepare_callbacks/checkpoint'
        checkpoint_model_dir = os.path.dirname(
            Path(config.checkpoint_model_filepath)
        )
        create_directories(
            paths_of_directories=[
                Path(config.root_dir),
                Path(config.tensorboard_root_log_dir),
                Path(checkpoint_model_dir)
            ]
        )
        
        # Creating and returning `PrepareCallbacksConfig`
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config

## Components

In [4]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig) -> None:
        """Inits PrepareCallbacks.

        Args:
            config (PrepareCallbacksConfig): The PrepareCallbacksConfig.
        """
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
        tb_running_log_dir = Path(os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_time_{timestamp}",
        ))
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_checkpoint_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True,
        )
        
    def get_tb_and_checkpoint_callbacks(self) -> list:
        return [
            self._create_tb_callbacks,
            self._create_checkpoint_callbacks
        ]

In [5]:
try:
    config = ConfigurationManager()

    prepare_callbacks_config = config.get_prepare_callbacks_config()

    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    prepare_callbacks.get_tb_and_checkpoint_callbacks()
except Exception as e:
    raise e